<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.62
    Uninstalling yfinance-0.2.62:
      Successfully uninstalled yfinance-0.2.62
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-11 14:12:00
-------------------
qualified stocks: 90
with latest results: 29
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  4.16 L
Current:  1.36 C
-------------------
Today PnL: -19.60 K (-0.14%)
Current PnL: -14.21 L (-9.9%)
CY Booked + Current PnL: -8.13 L (-5.66%)
-------------------
Total profit:  3.87 L
Total loss:  -18.08 L
-------------------
Total Booked + Current PnL: 18.85 L (16.18%)
Total Booked PnL: 33.06 L (28.37%)
Curr Year Booked PnL: 6.08 L (4.49%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.05 C
Est FTT PnL: 69.72 L (51.45%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.67%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,785.05,983.0,-2.96,H-LC,86.67,238266.0,43574.0,5528.0,1.40,22.38,2.32,25.21,34.0,7.88,1.81,31.75,XY25,NTT,INSURANCE
59,RELIANCE,1291.83,1526.0,6.50,X-LC,28.89,158322.0,17513.0,8011.0,0.97,12.44,5.06,18.13,19.0,2.19,1.21,25.01,XY25,NTT,REFINERIES
3,ADANIPORTS,1103.69,1583.0,2.87,M-LC,53.33,103085.0,24723.0,9309.0,-1.49,31.55,9.03,43.43,77.0,2.66,0.78,37.88,XY24,NTT,MISC
81,UJJIVANSFB,52.77,53.0,54.03,M-SC,82.22,133677.0,-8802.0,9424.0,-3.58,-6.18,7.05,0.44,241.0,-0.93,1.02,57.78,OX40N,NTT,BANKS
75,TATAELXSI,7332.28,7332.0,-11.95,X-MC,60.00,92505.0,-10147.0,10139.0,-0.34,-9.88,10.96,-0.00,55.0,-1.00,0.70,40.62,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,NATIONALUM,189.63,247.44,-36.43,L-SC,72.22,100488.0,-16.0,30659.0,-1.10,-0.02,30.51,30.49,246.0,-0.00,0.77,32.38,X5K,ATH,METALS
33,HINDALCO,651.95,761.55,-12.54,M-LC,23.33,104800.0,488.0,17051.0,-0.51,0.47,16.27,16.81,71.0,0.03,0.80,16.59,X5K,ATH,METALS
30,HAVELLS,1588.02,2069.16,-4.90,X-LC,16.67,245825.0,-1906.0,76968.0,0.40,-0.77,31.31,30.30,50.0,-0.02,1.87,12.42,X40,ATH,ELECTRICAL
5,APOLLOHOSP,6904.43,8285.00,-16.22,H-LC,15.56,103455.0,-111.0,20815.0,-0.31,-0.11,20.12,20.00,32.0,-0.01,0.79,13.95,X40N,BTT,HEALTHCARE


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,VIPIND,488.80,489.00,-938.04,H-SC,88.89,79814.0,-15502.0,15540.0,10.52,-16.26,19.47,0.04,151.0,-1.00,0.61,57.42,OX40N,NTT,MISC
84,VALIANTORG,512.64,838.00,-395.70,H-SC,100.00,125749.0,-7537.0,92136.0,4.99,-5.66,73.27,63.47,144.0,-0.08,0.96,110.79,XR,NTT,CHEMICALS
63,SAMMAANCAP,170.35,326.00,-164.07,M-SC,54.44,77820.0,-24390.0,117781.0,2.94,-23.86,151.35,91.37,199.0,-0.21,0.59,26.33,XY25,NTT,FINANCE
31,HCLTECH,1647.60,1959.14,19.14,H-LC,37.78,104615.0,4111.0,14897.0,2.81,4.09,14.24,18.91,31.0,0.28,0.80,26.25,X40,ATH,IT
27,GREENPANEL,375.16,537.00,204.26,M-SC,91.11,135932.0,-47146.0,126118.0,2.05,-25.75,92.78,43.14,210.0,-0.37,1.03,27.86,XY24,NTT,MISC


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UJJIVANSFB,52.77,53.00,54.03,M-SC,82.22,133677.0,-8802.0,9424.0,-3.58,-6.18,7.05,0.44,241.0,-0.93,1.02,57.78,OX40N,NTT,BANKS
0,5PAISA,564.75,565.00,153.61,H-SC,71.11,136935.0,-49997.0,50077.0,-2.74,-26.75,36.57,0.04,148.0,-1.00,1.04,31.00,OX40N,NTT,FINANCE
35,HINDZINC,514.80,744.74,39.23,M-LC,90.00,104300.0,1340.0,44651.0,-2.35,1.30,42.81,44.67,68.0,0.03,0.79,34.60,X5K,ATH,METALS
11,BANKINDIA,116.91,190.00,-28.22,M-MC,73.33,190328.0,10520.0,101902.0,-2.00,5.85,53.54,62.52,168.0,0.10,1.45,36.97,XR,NTT,BANKS
37,ICICIGI,1839.64,2260.25,-12.91,H-MC,48.89,143508.0,7375.0,23751.0,-1.94,5.42,16.55,22.86,33.0,0.31,1.09,18.71,X40,ATH,INSURANCE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UJJIVANSFB,52.77,53.0,54.03,M-SC,82.22,133677.0,-8802.0,9424.0,-3.58,-6.18,7.05,0.44,241.0,-0.93,1.02,57.78,OX40N,NTT,BANKS
73,SYMPHONY,1306.42,1306.0,-27.28,M-SC,3.33,155025.0,-16116.0,16061.0,-0.12,-9.42,10.36,-0.03,189.0,-1.00,1.18,11.42,OX40N,NTT,DURABLES
75,TATAELXSI,7332.28,7332.0,-11.95,X-MC,60.00,92505.0,-10147.0,10139.0,-0.34,-9.88,10.96,-0.00,55.0,-1.00,0.70,40.62,OX40N,NTT,IT
47,KANSAINER,299.63,340.0,-67.49,H-SC,7.78,226035.0,-43632.0,79971.0,-1.06,-16.18,35.38,13.47,143.0,-0.55,1.72,13.14,XY24,NTT,PAINTS
86,VIPIND,488.80,489.0,-938.04,H-SC,88.89,79814.0,-15502.0,15540.0,10.52,-16.26,19.47,0.04,151.0,-1.00,0.61,57.42,OX40N,NTT,MISC


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TCS,3794.03,4998.00,-16.80,X-LC,8.89,270910.0,-25024.0,118929.0,0.30,-8.46,43.90,31.73,1.0,-0.21,2.06,8.43,X40,BTT,IT
42,INFY,1461.46,2275.00,-9.55,X-LC,26.67,289658.0,28057.0,117572.0,1.39,10.72,40.59,55.67,2.0,0.24,2.21,17.48,X40,BTT,IT
51,LTIM,5564.16,7197.33,0.70,X-LC,62.22,193824.0,-6486.0,65280.0,-0.02,-3.24,33.68,29.35,3.0,-0.10,1.48,33.90,X200,ATH,IT
6,ASIANPAINT,2961.56,4250.00,-24.43,X-LC,6.67,150185.0,-51201.0,138816.0,-0.46,-25.42,92.43,43.51,5.0,-0.37,1.14,4.54,X40,BTT,PAINTS
34,HINDUNILVR,2413.81,2723.00,-15.01,X-LC,18.33,266739.0,-3608.0,38224.0,-0.57,-1.33,14.33,12.81,6.0,-0.09,2.03,10.39,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,UNIONBANK,123.87,163.0,-1.84,M-LC,87.78,173051.0,32211.0,12287.0,-1.37,22.87,7.10,31.59,88.0,2.62,1.32,50.32,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.87,M-LC,53.33,103085.0,24723.0,9309.0,-1.49,31.55,9.03,43.43,77.0,2.66,0.78,37.88,XY24,NTT,MISC
50,LICI,785.05,983.0,-2.96,H-LC,86.67,238266.0,43574.0,5528.0,1.40,22.38,2.32,25.21,34.0,7.88,1.81,31.75,XY25,NTT,INSURANCE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,130.64,228.00,101.81,M-MC,74.44,229060.0,4098.0,163549.0,0.07,1.82,71.40,74.53,183.0,0.03,1.74,32.99,XY24,BTT,STEEL
35,HINDZINC,514.80,744.74,39.23,M-LC,90.00,104300.0,1340.0,44651.0,-2.35,1.30,42.81,44.67,68.0,0.03,0.79,34.60,X5K,ATH,METALS
17,CAMS,3643.00,5250.99,4.89,H-SC,80.00,117933.0,15929.0,29094.0,-1.15,15.62,24.67,44.14,125.0,0.55,0.90,34.39,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.87,M-LC,53.33,103085.0,24723.0,9309.0,-1.49,31.55,9.03,43.43,77.0,2.66,0.78,37.88,XY24,NTT,MISC
32,HEROMOTOCO,4311.81,6039.03,1.43,H-MC,67.78,154906.0,3993.0,56463.0,1.07,2.65,36.45,40.06,101.0,0.07,1.18,26.70,AR,ATH,AUTO


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,WHIRLPOOL,1167.49,2270.00,-32.90,M-SC,44.44,195804.0,24183.0,137885.0,1.05,14.09,70.42,94.43,191.0,0.18,1.49,44.52,XR,NTT,DURABLES
89,WIPRO,243.46,420.00,-6.08,M-LC,25.56,160735.0,9790.0,99672.0,1.72,6.49,62.01,72.51,70.0,0.10,1.22,11.08,XR,NTT,IT
11,BANKINDIA,116.91,190.00,-28.22,M-MC,73.33,190328.0,10520.0,101902.0,-2.00,5.85,53.54,62.52,168.0,0.10,1.45,36.97,XR,NTT,BANKS
39,INDIAMART,2327.09,4871.06,-49.41,H-SC,58.89,131853.0,8517.0,126315.0,-0.04,6.91,95.80,109.32,122.0,0.07,1.00,30.26,AR,ATH,MISC
32,HEROMOTOCO,4311.81,6039.03,1.43,H-MC,67.78,154906.0,3993.0,56463.0,1.07,2.65,36.45,40.06,101.0,0.07,1.18,26.70,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.00,-51.71,L-SC,93.33,70061.0,-43488.0,83541.0,0.72,-38.30,119.24,35.27,260.0,-0.52,0.53,71.48,XR,NTT,HOTELS
4,ALKYLAMINE,2347.98,4567.52,-7.04,X-SC,85.56,86568.0,-14395.0,109837.0,-0.02,-14.26,126.88,94.53,59.0,-0.13,0.66,28.34,SR,ATH,CHEMICALS
52,MASFIN,326.60,399.50,-20.96,H-SC,68.89,90000.0,-7980.0,29853.0,-0.61,-8.14,33.17,22.32,138.0,-0.27,0.69,30.53,XR,ATH,FINANCE
3,ADANIPORTS,1103.69,1583.00,2.87,M-LC,53.33,103085.0,24723.0,9309.0,-1.49,31.55,9.03,43.43,77.0,2.66,0.78,37.88,XY24,NTT,MISC
35,HINDZINC,514.80,744.74,39.23,M-LC,90.00,104300.0,1340.0,44651.0,-2.35,1.30,42.81,44.67,68.0,0.03,0.79,34.60,X5K,ATH,METALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VALIANTORG,512.64,838.00,-395.70,H-SC,100.00,125749.0,-7537.0,92136.0,4.99,-5.66,73.27,63.47,144.0,-0.08,0.96,110.79,XR,NTT,CHEMICALS
7,ASIANTILES,75.41,137.00,7005.56,M-SC,98.89,77986.0,-15824.0,92445.0,-1.17,-16.87,118.54,81.67,235.0,-0.17,0.59,51.02,XR,NTT,CERAMICS
46,JPPOWER,18.73,26.20,-33.07,L-SC,97.78,135784.0,-6058.0,62624.0,-0.33,-4.27,46.12,39.88,256.0,-0.10,1.03,37.61,XY24,NTT,POWER
74,TANLA,917.30,1963.11,-33.79,H-SC,96.67,161898.0,-77517.0,350477.0,-1.41,-32.38,216.48,114.01,127.0,-0.22,1.23,48.49,AR,ATH,IT
78,TITAGARH,1097.23,1548.00,-7.53,H-SC,95.56,162461.0,-29554.0,108443.0,-1.66,-15.39,66.75,41.08,152.0,-0.27,1.24,37.50,XY24,NTT,ENGINEERING


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.90
1,25,43.75
2,50,72.58


In [21]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     32.28
XY25     15.49
XR       11.06
X40      11.04
OX40N     8.56
X40N      8.48
AR        8.04
X5K       2.36
X200      1.48
SR        1.17
Name: CurrAlloc%, dtype: float64

In [22]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    18.99
X-LC    18.30
M-SC    14.41
M-LC    10.19
H-MC     9.06
H-LC     7.91
X-MC     6.24
M-MC     5.86
L-SC     3.28
X-SC     2.82
L-LC     1.13
Name: CurrAlloc%, dtype: float64

In [23]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.23,-1.15,34.80
IT,11.27,-12.14,69.07
BANKS,8.65,-7.14,50.32
MISC,6.61,-24.26,72.85
FINANCE,6.45,-18.64,64.29
PAINTS,5.68,-16.88,39.92
INSURANCE,5.27,3.82,24.92
ELECTRICAL,5.20,-2.93,47.58
AUTO,4.79,-7.57,54.87


In [24]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2688949.0,25
XR,1107647.0,13
AR,1013620.0,9
X40,597741.0,8
XY25,495224.0,10
X40N,414881.0,8
OX40N,305614.0,11
SR,191196.0,2
X5K,92361.0,3


In [25]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1871348.0,18
M-SC,1433222.0,18
X-LC,809612.0,11
M-MC,573170.0,5
H-MC,536487.0,7
M-LC,375473.0,8
X-MC,324929.0,5
L-SC,298220.0,5
X-SC,262392.0,4


In [26]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,891151.0,6
M-SC,XY24,569209.0,5
H-SC,AR,531394.0,3
X-LC,X40,452285.0,4
M-MC,XY24,386118.0,3
M-SC,XR,342550.0,4
H-MC,XY24,220091.0,2
M-LC,XY24,201113.0,4
H-SC,XR,194299.0,3


In [27]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
